In [555]:
sigma_black = [0.3338,
0.3338,
0.3338,
0.395311485,
0.428180495,
0.462400766,
0.497098782,
0.456725677,
0.46998663,
0.48326345,
0.49709287,
0.478444649,
0.485478514]

In [556]:
sigma_black_square = [sigma**2 for sigma in sigma_black]

In [557]:
sigma_black_square

[0.11142243999999998,
 0.11142243999999998,
 0.11142243999999998,
 0.15627117017290523,
 0.18333853629844502,
 0.21381446839738677,
 0.24710719906588355,
 0.20859834403110833,
 0.2208874323787569,
 0.2335435621059025,
 0.24710132140483693,
 0.2289092821567332,
 0.23568938755564822]

In [533]:
import numpy as np
from scipy.integrate import quad
from scipy.optimize import fsolve,minimize
import pandas as pd

In [37]:
T = [0.25 * i for i in range(1, 15)]

In [44]:
T

[0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0, 2.25, 2.5, 2.75, 3.0, 3.25, 3.5]

In [558]:
func7 = lambda x,a,b,c,d,T: ((b*(T-x)+a)*np.exp(-c*(T-x))+d)**2

In [559]:
def calibrate_vols_func7(params):
    a,b,c,d = params
    phi = [1 for _ in range(13)]
    ret = 0
    for i in range(13):
        Ti = T[i]
        integral,err = quad(func7, 0, Ti, args=(a,b,c,d,Ti))
        ret += (sigma_black_square[i] - integral*phi[i]**2/Ti)**2
    print(ret)
    return ret

In [566]:
# Give a,b,c,d initials, calibrate a,b,c,d using function 7
initial_guess = [0.5,0.5,0.5,0.5]
# initial_guess = [0.5 for _ in range(13+4)]
result = minimize(calibrate_vols_func7, initial_guess, method = 'L-BFGS-B', bounds=[(0, None), (0, None), (None, None), (None,None)])
if result.success:
    fitted_params = result.x
    print(fitted_params)
else:
    print(result.x)
    raise ValueError(result.message)

11.579735186626017
11.579735547113307
11.579735417746816
11.57973493410258
11.579735708771109
0.46052621341193756
0.4605262116608716
0.4605262142331558
0.4605262117377269
0.46052621502281976
0.45847602973478196
0.4584760281380525
0.45847603066591053
0.4584760279439475
0.45847603172400886
0.44856820670715
0.448568205718867
0.4485682080607092
0.4485682044655578
0.4485682101838554
0.38393937697964414
0.3839393781968818
0.38393937969249337
0.3839393732625196
0.3839393857138782
0.061380470427699854
0.06138047198619126
0.06138047077126793
0.061380470376390134
0.061380470575019816
0.0632569292363267
0.06325693209379123
0.06325693081302432
0.06325692754312486
0.06325693530369783
0.058777192211465705
0.05877719435062304
0.05877719311087073
0.05877719138168741
0.05877719504670202
0.025238937979436098
0.025238939952448115
0.02523893865887698
0.025238937516088765
0.025238940600588143
0.2687139317681574
0.268713938366558
0.26871393525710496
0.2687139317681574
0.2687139486898869
0.025192305518623746

In [567]:
a,b,c,d = fitted_params

In [568]:
# calibrated values
print(a,b,c,d)

0.3011066205989805 2.546296175696699e-07 1.6758218921693948 -0.5464782535549924


In [ ]:
a,b,c,d = 

In [569]:
def get_phi_square(a,b,c,d):
    phi_square = []
    for i in range(13):
        Ti = T[i]
        integral,err = quad(func7, 0, Ti, args=(a,b,c,d,Ti))
        phi_square += sigma_black_square[i]*Ti / integral,
    return phi_square

In [570]:
# using calibrated value calculate phi^2 for T from 0.25 to 3.5
phi_square = get_phi_square(a,b,c,d)

In [577]:
phi_square

[1.2220603204490141,
 0.9303097974197986,
 0.7713455221851302,
 0.9464327937264954,
 1.0063204298024981,
 1.0895852546981435,
 1.1890923850465769,
 0.9595271087066429,
 0.9800411469851094,
 1.0062412614637124,
 1.0392218940324405,
 0.9434715747667429,
 0.9549895899118719]

In [571]:
# get phi values
phi = [phi**(0.5) for phi in phi_square]

In [572]:
# add a dummy head for T = 0, so value of phi[1] is at time T0, phi[2] is at time T1
phi = [0] + phi

In [573]:
phi

[0,
 1.1054683715281113,
 0.964525685204805,
 0.8782627865195759,
 0.9728477752076609,
 1.0031552371405426,
 1.0438320050171597,
 1.090455127479612,
 0.9795545460599133,
 0.9899702758088798,
 1.0031157766996353,
 1.019422333496986,
 0.9713246495208195,
 0.9772356880056479]

In [582]:
for i in range(13):
    Ti = T[i]
    integral, err = quad(func7, 0, Ti, args=(a,b,c,d,Ti))
    print(phi_square[i] / Ti * integral, sigma_black_square[i], abs(phi_square[i] / Ti * integral - sigma_black_square[i]))

0.11142243999999998 0.11142243999999998 0.0
0.11142243999999998 0.11142243999999998 0.0
0.11142243999999998 0.11142243999999998 0.0
0.15627117017290523 0.15627117017290523 0.0
0.183338536298445 0.18333853629844502 2.7755575615628914e-17
0.21381446839738677 0.21381446839738677 0.0
0.24710719906588355 0.24710719906588355 0.0
0.20859834403110833 0.20859834403110833 0.0
0.22088743237875688 0.2208874323787569 2.7755575615628914e-17
0.2335435621059025 0.2335435621059025 0.0
0.24710132140483693 0.24710132140483693 0.0
0.2289092821567332 0.2289092821567332 0.0
0.2356893875556482 0.23568938755564822 2.7755575615628914e-17


In [574]:
# get expected sigma
expected_sigma = []

In [575]:
for i in range(13):
    Ti = T[i]
    sigma_ = -phi[i+1]*((b*(Ti)+a)*np.exp(-c*(Ti))+d)
    print(sigma_)
    expected_sigma += sigma_,

0.3851786827660224
0.40145050160562773
0.4047035224429361
0.4768168294459439
0.5110200159215967
0.544983637156525
0.5784244999529076
0.5249740934226914
0.5341298196329878
0.5436040539808177
0.5540328164338215
0.5288905195473628
0.532769319298739


In [235]:
sigma = lambda x,a,b,c,d,T,phi: phi*((b*(T-x)+a)*np.exp(-c*(T-x))+d)

In [75]:
T0s = [0.25,
0.25,
0.25,
0.5,
0.5,
0.5,
0.75,
0.75,
1,
1,
2]

In [76]:
Tns = [1.25,
2.25,
3.25,
1.5,
2.5,
3.5,
1.75,
2.75,
2,
3,
3]

In [77]:
VOLS = [0.4315,
0.4177,
0.527,
0.387,
0.474,
0.4864,
0.4555,
0.4624,
0.4432,
0.4496,
0.4559
]

In [295]:
# swaption data
swaptions = [(T0s[i], Tns[i], VOLS[i]) for i in range(len(VOLS))]

In [296]:
swaptions

[(0.25, 1.25, 0.4315),
 (0.25, 2.25, 0.4177),
 (0.25, 3.25, 0.527),
 (0.5, 1.5, 0.387),
 (0.5, 2.5, 0.474),
 (0.5, 3.5, 0.4864),
 (0.75, 1.75, 0.4555),
 (0.75, 2.75, 0.4624),
 (1, 2, 0.4432),
 (1, 3, 0.4496),
 (2, 3, 0.4559)]

In [83]:
# No usage
sigma_T0_dict = {T[i]:sigma_black_square[i] for i in range(len(sigma_black_square))}

In [84]:
sigma_T0_dict

{0.25: 9.416919689999999e-08,
 0.5: 8.864374836100001e-08,
 0.75: 1.26039910441e-07,
 1.0: 2.6832503600100004e-07,
 1.25: 3.94033653841e-07,
 1.5: 5.00116281721e-07,
 1.75: 6.16488788224e-07,
 2.0: 8.94182054544e-07,
 2.25: 8.864297820160001e-07,
 2.5: 1.0546037097209998e-06,
 2.75: 1.2278922586090002e-06,
 3.0: 1.3721381326890001e-06,
 3.25: 1.3873400453160002e-06}

In [95]:
# Range T0, T1 ... T13
P0 = [0.995096717,
0.990532881,
0.986431072,
0.982467879,
0.978587113,
0.97485501,
0.971226926,
0.967690877,
0.963842966,
0.960300259,
0.956696327,
0.953036058,
0.949370743,
0.945857254]

In [74]:
# Range 0, T0, T1 ... T13
F = [0.0192813,
0.018227298,
0.016450148,
0.01578489,
0.015517885,
0.01531347,
0.014778073,
0.014298691,
0.015289503,
0.015092042,
0.014902652,
0.014708837,
0.01527343,
0.015196126,
0.015121288]

In [129]:
def get_T_index(val):
    return int(val/0.25 - 1)

In [117]:
# No usage
def generate_w_matrix():
    w_matrix = [[0 for _ in range(13)] for _ in range(13)]
    tau = 0.25
    for i in range(13):
        numerator = tau * P0[i]
        denominator = 0.0
        for j in range(13):
            denominator += tau * P0[j]
            w_matrix[i][j] = numerator / denominator
    return w_matrix

In [179]:
# tau times p for T0, T1,..., T13
tau_times_p = [
0,
0.2503847,
0.249347854,
0.251075125,
0.250083373,
0.243713753,
0.245504584,
0.24729878,
0.251670108,
0.234740063,
0.241831572,
0.248848304,
0.239979827,
0.231209551
]

In [1247]:
len(tau_times_p)

14

In [283]:
def get_wi(i, T0, Tn): #From T0 to Tn-1
    numerator = tau_times_p[i]
    denominator = 0.0
    for j in range(T0, Tn):
        denominator += tau_times_p[j]
    return numerator / denominator

In [1250]:
def generate_S_matrix():
    s_matrix = [[0 for _ in range(14)] for _ in range(14)]
#     for i in range(len(T0s)):
    for i in range(len(T0s)):
        T0, Tn = T0s[i], Tns[i]
        t0_idx = get_T_index(T0)
        n = get_T_index(Tn)
        ret = 0.0
        sum_w = 0
        for j in range(t0_idx, n):
            w = get_wi(j+1, t0_idx+1, n+1)
            ret += w*F[j+1]
            sum_w += w
        print(t0_idx, n, ret, sum_w)
        s_matrix[t0_idx][n] = ret
    return s_matrix

In [1251]:
s_matrix = generate_S_matrix()
# swaption S_T0_Tn
# T0 Tn S_T0_Tn

0 4 0.016494906076760837 0.9999999999999999
0 8 0.015712680530331487 1.0
0 12 0.015476836764797662 1.0
1 5 0.01576901384369275 1.0
1 9 0.01531980668205354 1.0000000000000002
1 13 0.015220108090333478 1.0
2 6 0.015351879834534798 1.0
2 10 0.015125124360902962 0.9999999999999999
3 7 0.014977695553204749 0.9999999999999999
3 11 0.014988013579457907 0.9999999999999998
7 11 0.014998432035554612 1.0


In [70]:
integral_term = lambda x,a,b,c,d,Ti,Ti_1,Tj,Tj_1,phi_i,phi_j,alpha,beta,gamma: (alpha + (1-alpha)*np.exp(-abs(Ti - Tj)*(beta-gamma*max(Ti, Tj)))) * (phi_i*((b*(Ti_1-x)+a)*np.exp(-c*(Ti_1-x))+d)) * (phi_j*((b*(Tj_1-x)+a)*np.exp(-c*(Tj_1-x))+d))

In [327]:
integral_term_simple = lambda x,a,b,c,d,Ti,Ti_1,Tj,Tj_1,phi_i,phi_j,beta: (np.exp(-abs(Ti - Tj)*beta)) * (phi_i*((b*(Ti_1-x)+a)*np.exp(-c*(Ti_1-x))+d)) * (phi_j*((b*(Tj_1-x)+a)*np.exp(-c*(Tj_1-x))+d))

In [623]:
def calibrate_corr(params):
#     alpha, beta, gamma = params
#     a,b,c,d = known
    beta = params
    ret = 0
    print('-'*60)
#     print(alpha,beta,gamma)
    print(beta)
    for swaption in swaptions:
        T0, Tn, VOL = swaption
        t0_idx = get_T_index(T0)
        tn_idx = get_T_index(Tn)
        sigma_black_2 = VOL**2
        left_part = sigma_black_2 * T0
        right_part = 0
        for i in range(t0_idx+1, tn_idx+1): #i from index_T0+1 to index_Tn
            Ti, Ti_1 = T[i], T[i-1]
            phi_i = phi[i]
#             wi = get_wi(i, t0_idx, tn_idx+1) #get wi from index_T0 to index_Tn
            wi = get_wi(i, t0_idx+1, tn_idx+1)
#             wi = get_wi(i, 0, tn_idx+1)
#             wi = get_wi(i, t0_idx+1, i+1)
            Fi = F[i]
            for j in range(t0_idx+1, tn_idx+1): 
                Tj, Tj_1 = T[j], T[j-1]
                phi_j = phi[j]
#                 integral, err = quad(integral_term, 0, T0, args=(a,b,c,d,Ti,Ti_1,Tj,Tj_1,phi_i,phi_j,alpha,beta,gamma))
#                 print(i,j,t0_idx,n)
#                 print(Ti, Tj, (np.exp(-abs(Ti - Tj)*beta)))
                integral, err = quad(integral_term_simple, 0, T0, args=(a,b,c,d,Ti,Ti_1,Tj,Tj_1,phi_i,phi_j,beta))
#                 wj = get_wi(j, t0_idx, tn_idx+1) #get wj from index_T0 to index_Tn
                wj = get_wi(j, t0_idx+1, tn_idx+1) #get wj from index_T0 to index_Tn
#                 wj = get_wi(j, 0, tn_idx+1)
#                 wj = get_wi(j, t0_idx+1, j+1)
                Fj = F[j]
#                 print(wi*wj*Fi*Fj/(s_matrix[t0_idx][tn_idx]**2), integral)
                right_part += wi*wj*Fi*Fj/s_matrix[t0_idx][tn_idx]**2 * integral 
#         print(left_part, right_part)
#         print(left_part, right_part)
        ret += (left_part - right_part)**2
#         break
    print(ret)
    return ret

In [1252]:
# Calibrate beta
initial_guess = [0.3,0.5,0.5]
bounds = [(0, 1), (0, None), (0, None)]
# initial_guess = [0.5 for _ in range(13+4)]
initial_guess = [0.1]
bounds = [(0, None)]
result = minimize(calibrate_corr, initial_guess, method = 'L-BFGS-B', bounds=bounds)
if result.success:
    fitted_params = result.x
    print(fitted_params)
else:
    print(result.x)
    raise ValueError(result.message)

------------------------------------------------------------
[0.1]
0.006246747690522928
------------------------------------------------------------
[0.10000001]
0.006246747430974902
------------------------------------------------------------
[0.1259548]
0.005628935216894141
------------------------------------------------------------
[0.12595481]
0.005628934999930166
------------------------------------------------------------
[0.25819345]
0.00401364421438126
------------------------------------------------------------
[0.25819346]
0.004013644177529804
------------------------------------------------------------
[0.28524979]
0.003955396622370146
------------------------------------------------------------
[0.2852498]
0.003955396615837209
------------------------------------------------------------
[0.29107981]
0.003953422583507086
------------------------------------------------------------
[0.29107982]
0.003953422583253402
------------------------------------------------------------

In [624]:
corr_matrix = [[0 for i in range(14)] for j in range(14)]
cov_matrix = [[0 for i in range(13)] for j in range(13)]

In [1253]:
beta = fitted_params[0]

In [1254]:
beta

0.29131534207398035

In [626]:
# Get correlation matrix T0 to Tn
for i in range(14):
    Ti = T[i]
    for j in range(i, 14):
        Tj = T[j]
        corr_matrix[i][j] = np.exp(-beta*abs(Ti - Tj))
        corr_matrix[j][i] = np.exp(-beta*abs(Tj - Ti))

In [627]:
pd.DataFrame(corr_matrix)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1.000000,0.925323,0.856223,0.792283,0.733118,0.678371,0.627713,0.580837,0.537462,0.497326,0.460187,0.425822,0.394023,0.364599
1,0.925323,1.000000,0.925323,0.856223,0.792283,0.733118,0.678371,0.627713,0.580837,0.537462,0.497326,0.460187,0.425822,0.394023
2,0.856223,0.925323,1.000000,0.925323,0.856223,0.792283,0.733118,0.678371,0.627713,0.580837,0.537462,0.497326,0.460187,0.425822
3,0.792283,0.856223,0.925323,1.000000,0.925323,0.856223,0.792283,0.733118,0.678371,0.627713,0.580837,0.537462,0.497326,0.460187
4,0.733118,0.792283,0.856223,0.925323,1.000000,0.925323,0.856223,0.792283,0.733118,0.678371,0.627713,0.580837,0.537462,0.497326
5,0.678371,0.733118,0.792283,0.856223,0.925323,1.000000,0.925323,0.856223,0.792283,0.733118,0.678371,0.627713,0.580837,0.537462
6,0.627713,0.678371,0.733118,0.792283,0.856223,0.925323,1.000000,0.925323,0.856223,0.792283,0.733118,0.678371,0.627713,0.580837
7,0.580837,0.627713,0.678371,0.733118,0.792283,0.856223,0.925323,1.000000,0.925323,0.856223,0.792283,0.733118,0.678371,0.627713
8,0.537462,0.580837,0.627713,0.678371,0.733118,0.792283,0.856223,0.925323,1.000000,0.925323,0.856223,0.792283,0.733118,0.678371
9,0.497326,0.537462,0.580837,0.627713,0.678371,0.733118,0.792283,0.856223,0.925323,1.000000,0.925323,0.856223,0.792283,0.733118


In [729]:
# Get covariance matrix T0 to Tn-1
def update_cov_matrix(cov_matrix, n=0):
    for i in range(n+1, 14):
        Ti, Ti_1 = T[i], T[i-1]
        phi_i = phi[i]
        for j in range(i, 14):
            Tj, Tj_1 = T[j], T[j-1]
            phi_j = phi[j]
            integral, err = quad(integral_term_simple, Ti_1, Ti, args=(a,b,c,d,Ti,Ti_1,Tj,Tj_1,phi_i,phi_j,beta))
    #         integral, err = quad(integral_term, Ti_1, Ti, args=(a,b,c,d,Ti,Ti_1,Tj,Tj_1,phi_i,phi_j,alpha,beta,gamma))
            cov_matrix[i-1][j-1] = integral
            cov_matrix[j-1][i-1] = integral
# cov_matrix[0][0] is for Ti=T1, Ti_1=T0, Tj=T1, Tj_1=T0

In [1140]:
# Get covariance matrix T0 to Tn-1
def update_cov_matrix_v2(cov_matrix, n=0, t=13):
    for i in range(n+1, t+1):
        Ti, Ti_1 = T[i], T[i-1]
        phi_i = phi[i]
        for j in range(i, t+1):
            Tj, Tj_1 = T[j], T[j-1]
            phi_j = phi[j]
            integral, err = quad(integral_term_simple, Ti_1, Ti, args=(a,b,c,d,Ti,Ti_1,Tj,Tj_1,phi_i,phi_j,beta))
    #         integral, err = quad(integral_term, Ti_1, Ti, args=(a,b,c,d,Ti,Ti_1,Tj,Tj_1,phi_i,phi_j,alpha,beta,gamma))
            cov_matrix[i-1][j-1] = integral
            cov_matrix[j-1][i-1] = integral
# cov_matrix[0][0] is for Ti=T1, Ti_1=T0, Tj=T1, Tj_1=T0

In [662]:
from scipy.linalg import sqrtm, cholesky

In [632]:
# get square root of covariance matrix
sqrt_cov_mat = sqrtm(cov_matrix)

In [757]:
def getsubgrid(x, y, grid):
    return [item[x:] for item in grid[y:]]

In [711]:
# Forward at time 0 (not T0) from F0(0) to F13(0), F0(0) is not used
F0 = F[:-1]

In [712]:
# init a 14 x 14 matrix to store Fk, row = time, col = k
def init_Fk():
    ret = [F0]
    for _ in range(13):
        ret += [0 for _ in range(14)],
    return ret

In [1159]:
def init_Fk_v2(F0,t=13):
    ret = [F0]
    for _ in range(t):
        ret += [0 for _ in range(len(F0))],
    return ret

In [713]:
def get_mu_k_integral(k, i, F_mat, cov_matrix):
    ret = 0
    for j in range(k+1, 14): #k+1 to 13
        ret += (F_mat[i-1][j]*0.25/(1+F_mat[i-1][j]*0.25) + F_mat[i][j]*0.25/(1+F_mat[i][j])) * cov_matrix[j-1][k-1]
    return -0.5 * ret

In [1183]:
def get_mu_k_integral_v2(k, i, F_mat, cov_matrix, t):
    ret = 0
    for j in range(k+1, t): #k+1 to 13
        ret += (F_mat[j][i-1]*0.25/(1+F_mat[j][i-1]*0.25) + F_mat[j][i]*0.25/(1+F_mat[j][i])) * cov_matrix[k-1][j-1]
    return -0.5 * ret

In [1108]:
def get_sum_part(k, n, sqrt_cov_mat):
    ret = 0.0
    for i in range(n, 14):
        ret += sqrt_cov_mat[k-1][i-1] * np.random.normal(0, 1)
    return ret

In [1184]:
def get_sum_part_v2(k, n, sqrt_cov_mat, t):
    ret = 0.0
    for i in range(n, t):
        ret += sqrt_cov_mat[i-1][k-1] * np.random.normal(0, 1)
    return ret

In [1109]:
def get_exp_term(k, i, F_mat, cov_matrix, sqrt_cov_mat):
    mu_k_part = get_mu_k_integral(k, i, F_mat, cov_matrix)
    sum_part = get_sum_part(k, i, sqrt_cov_mat)
    return mu_k_part - 0.5 * cov_matrix[k-1][k-1] + sum_part

In [1185]:
def get_exp_term_v2(k, i, F_mat, cov_matrix, sqrt_cov_mat, t):
    mu_k_part = get_mu_k_integral_v2(k, i, F_mat, cov_matrix, t)
    sum_part = get_sum_part_v2(k, i, sqrt_cov_mat, t)
    return mu_k_part - 0.5 * cov_matrix[k-1][k-1] + sum_part

In [1110]:
def modify_Fk_at_Ti(i, F_mat, cov_matrix):
    tmp_cov_matrix = getsubgrid(i,i,cov_matrix)
    tmp_sqrt_cov_mat = sqrtm(tmp_cov_matrix)
    sqrt_cov_mat = [[0 for i in range(13)] for j in range(13)]
    for m in range(len(tmp_sqrt_cov_mat)):
        for n in range(len(tmp_sqrt_cov_mat)):
            sqrt_cov_mat[i+m][i+n] = tmp_sqrt_cov_mat[m][n]
    i = i + 1
    for k in range(i, 14):
        F_mat[i][k] = F_mat[i-1][k] * np.exp(get_exp_term(k, i, F_mat, cov_matrix, sqrt_cov_mat))

In [1180]:
def modify_Fk_at_Ti_v2(i, F_mat, cov_matrix, t):
    tmp_cov_matrix = getsubgrid(i,i,cov_matrix)
    tmp_sqrt_cov_mat = sqrtm(tmp_cov_matrix)
    sqrt_cov_mat = [[0 for i in range(t)] for j in range(t)]
    for m in range(len(tmp_sqrt_cov_mat[0])):
        for n in range(len(tmp_sqrt_cov_mat)):
            sqrt_cov_mat[i+n][i+m] = tmp_sqrt_cov_mat[n][m]
    i = i + 1
    for k in range(i, len(F_mat[0])):
        F_mat[k][i] = F_mat[k][i-1] * np.exp(get_exp_term_v2(k, i, F_mat, cov_matrix, sqrt_cov_mat, len(F_mat[0])))

In [1111]:
def simulate_Fk():
    F_mat = init_Fk()
    cov_matrix = [[0 for i in range(13)] for j in range(13)]
    for i in range(0, 13):
        update_cov_matrix(cov_matrix, i)
        modify_Fk_at_Ti(i, F_mat, cov_matrix)
    return F_mat

In [1181]:
def simulate_Fk_v2(F0, t):
    F_mat = init_Fk_v2(F0, t)
    cov_matrix = [[0 for i in range(t)] for j in range(t)]
    for i in range(0, t):
        print("SimFK", i)
        update_cov_matrix_v2(cov_matrix, i, t)
        modify_Fk_at_Ti_v2(i, F_mat, cov_matrix, t)
        print(pd.DataFrame(F_mat))
    return F_mat

In [1186]:
simulate_Fk_v2(F0[:7], 10)

SimFK 0
           0                 1        2         3         4         5  \
0   0.019281  (0.018227298+0j)  0.01645  0.015785  0.015518  0.015313   
1   0.000000                0j  0.00000  0.000000  0.000000  0.000000   
2   0.000000                0j  0.00000  0.000000  0.000000  0.000000   
3   0.000000                0j  0.00000  0.000000  0.000000  0.000000   
4   0.000000                0j  0.00000  0.000000  0.000000  0.000000   
5   0.000000                0j  0.00000  0.000000  0.000000  0.000000   
6   0.000000                0j  0.00000  0.000000  0.000000  0.000000   
7   0.000000                0j  0.00000  0.000000  0.000000  0.000000   
8   0.000000                0j  0.00000  0.000000  0.000000  0.000000   
9   0.000000                0j  0.00000  0.000000  0.000000  0.000000   
10  0.000000                0j  0.00000  0.000000  0.000000  0.000000   

           6  
0   0.014778  
1   0.000000  
2   0.000000  
3   0.000000  
4   0.000000  
5   0.000000  
6   0.0000

[[0.0192813,
  0.018227298,
  0.016450148,
  0.01578489,
  0.015517885,
  0.01531347,
  0.014778073],
 [0, 0j, 0, 0, 0, 0, 0],
 [0, 0j, 0j, 0, 0, 0, 0],
 [0, 0j, 0j, 0j, 0, 0, 0],
 [0, 0j, 0j, 0j, 0j, 0, 0],
 [0, 0j, 0j, 0j, 0j, 0j, 0],
 [0, 0j, 0j, 0j, 0j, 0j, 0j],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0]]

In [1116]:
rate1 = 0.022
rate2 = 0.025

In [1328]:
def get_P(Ti, Tj, F):
    if Ti == Tj: return 1
    prod = 1
    for k in range(Ti, Tj+1):
#         print(k, Ti, F[Ti][k])
        prod *= (1 + F[Ti][k]*0.25)
    return (1.0/prod).real

In [1118]:
def simulate_P_and_S(F_mat):
    p_mat = [[0 for i in range(14)] for j in range(14)]
    s_mat = [0 for i in range(14)]
    for i in range(13):
        sum_p = 0
        for j in range(i+1, 14):
            p_mat[i][j] = get_P(i+1, j, F_mat)
#             print(i,j,p_mat[i][j])
            sum_p += p_mat[i][j]*0.25
        s_mat[i] = (1.0 - p_mat[i][-1]) / sum_p
    return p_mat, s_mat

In [1119]:
from sklearn.linear_model import LinearRegression

In [1120]:
def get_coef_LR(X,Y):
#     print(X)
#     print(Y)
    X = np.array(X)
    Y = np.array(Y)
    reg = LinearRegression().fit(X, Y)
#     print(reg.intercept_, reg.coef_)
    ret = list([reg.intercept_]) + list(reg.coef_)
    return ret

In [1125]:
def monte_carlo_simulation(n=1):
    monte_carlo_cache = {}
    for i in range(n):
        monte_carlo_cache[i] = {}
        simulated_Fk = simulate_Fk()
        simulated_P, simulated_S = simulate_P_and_S(simulated_Fk)
#         print(pd.DataFrame(simulated_P))
        monte_carlo_cache[i]['Fk'] = simulated_Fk.copy()
        monte_carlo_cache[i]['P'] = simulated_P.copy()
        monte_carlo_cache[i]['S'] = simulated_S.copy()
        monte_carlo_cache[i]['Q'] = [1012.5].copy()
    for t in range(13, 0, -1):
        r = rate1 if t < 10 else rate2
        delta_t = 0.5 if t%2 == 0 else 0.25
        coupon = 12.5 if t%2 == 0 else 0
        cont_list = []
        call_list = []
        X = []
        for i in range(n):
            # TODO: Calculate Q12 CV and Call Value from all path
#             print(monte_carlo_cache[i]['P'][t-1][-1])
            cont_val = monte_carlo_cache[i]['P'][t-1][-1] * monte_carlo_cache[i]['Q'][-1] + coupon
            cont_list += cont_val,
            call_val = 1000 * (1+r*delta_t)
            call_list += call_val,
            x1 = monte_carlo_cache[i]['Fk'][t][t].real
            x2 = monte_carlo_cache[i]['S'][t-1]
            X += [x1, x2, x1**2, x2**2, x1*x2],
        # TODO: Linear Regression get coefficients
        b0, b1, b2, b3, b4, b5 = get_coef_LR(X, cont_list)
#         print("Time {}, b0: {}, b1: {}, b2: {}, b3: {}, b4: {}, b5: {}".format(t, b0, b1, b2, b3, b4, b5))
        for i in range(n):
            # TODO: Recalculate VC for all path and compare with Call Value
            cont_val_hat = b0 + b1*X[i][0] + b2*X[i][1] + b3*X[i][2] + b4*X[i][3] + b5*X[i][4]
            call_val = call_list[i]
            monte_carlo_cache[i]['Q'].append((call_val if call_val < cont_val_hat else cont_list[i]) / monte_carlo_cache[i]['P'][t-1][-1])
    return monte_carlo_cache

In [1255]:
mcs = monte_carlo_simulation(30)

In [1256]:
ret = 0
n = len(mcs)
p_t0_t13 = 0.945857254
for i in range(n):
#     print(mcs[i]['Q'][-1], mcs[i]['P'][0][-1])
    print(mcs[i]['Q'][-1], p_t0_t13)
    ret +=  mcs[i]['Q'][-1]*p_t0_t13 * 1.0/n
print(ret)

1057.0522261032338 0.945857254
1057.4369391406112 0.945857254
1053.914806734585 0.945857254
1058.749497131132 0.945857254
1054.5332583815139 0.945857254
1058.2933219064423 0.945857254
1056.6225508170935 0.945857254
1056.8054707217377 0.945857254
1058.8273846699867 0.945857254
1056.3848341015569 0.945857254
1058.9175819591662 0.945857254
1058.24864458685 0.945857254
1056.3384490274502 0.945857254
1055.537422785486 0.945857254
1057.223186325063 0.945857254
1054.405770690277 0.945857254
1054.6399441183476 0.945857254
1057.7689604297925 0.945857254
1055.9470194111511 0.945857254
1058.3985409842933 0.945857254
1056.195407511047 0.945857254
1057.7905070365734 0.945857254
1056.7071309201096 0.945857254
1055.8447379404765 0.945857254
1057.1716360323599 0.945857254
1056.9847854187879 0.945857254
1056.1530836667505 0.945857254
1053.997222600402 0.945857254
1054.8388563078179 0.945857254
1053.9035790233618 0.945857254
999.31813876133


In [1188]:
def format_print(l):
    print(pd.DataFrame(l))

In [1329]:
pd.DataFrame(mcs[0]['P'])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0,0.994981,0.991534,0.987362,0.983699,0.980205,0.976310,0.973033,0.969611,0.966038,0.962315,0.958674,0.954826,0.951230
1,0,0.000000,0.997009,0.992614,0.988468,0.985582,0.981155,0.977472,0.973340,0.969879,0.966675,0.962740,0.958989,0.955273
2,0,0.000000,0.000000,0.995150,0.991866,0.988693,0.984141,0.980952,0.977513,0.973965,0.971095,0.966840,0.962706,0.958643
3,0,0.000000,0.000000,0.000000,0.997126,0.994241,0.989721,0.986914,0.984110,0.980830,0.977594,0.972865,0.967891,0.962880
4,0,0.000000,0.000000,0.000000,0.000000,0.996660,0.991895,0.988070,0.984792,0.981287,0.978209,0.973640,0.968642,0.963754
5,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.995493,0.992568,0.988918,0.984336,0.981702,0.975941,0.971932,0.967082
6,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.995924,0.992321,0.988039,0.985765,0.981238,0.977479,0.971755
7,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.997114,0.993289,0.990457,0.986382,0.982410,0.976918
8,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.996337,0.993515,0.989186,0.985319,0.980840
9,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.996665,0.991637,0.987712,0.983945


In [1279]:
swaption_price = 0
n = len(mcs)
P0_T4 = 0.978587113
K_ = 0.016769
for i in range(n):
    P_T0_T4 = mcs[i]['P'][0][4]
    sum_part = sum([K_*0.25*mcs[i]['P'][0][j] for j in range(1,5)])
    V = P0_T4 * max(0, 1-P_T0_T4-sum_part)/P_T0_T4
    print(P_T0_T4, sum_part, 1-P_T0_T4-sum_part)
    swaption_price +=  V * 1.0/n
print(swaption_price)

0.9836990802022262 0.016591151074751724 -0.0002902312769779078
0.9836588969537535 0.01659251817962604 -0.0002514151333795657
0.9845550878255712 0.016603981284481047 -0.0011590691100522646
0.984407114771202 0.016605662463924156 -0.001012777235126175
0.983861735006226 0.016601882735612743 -0.0004636177418387219
0.9843455957708345 0.01660970174170702 -0.0009552975125414892
0.9845915372426216 0.016599181979177507 -0.0011907192217991136
0.9838062771136015 0.016590884848789737 -0.00039716196239120496
0.9824597639418626 0.01657565822650324 0.0009645778316341308
0.9838683723154477 0.01659311624189113 -0.0004614885573387892
0.9822663495765183 0.016583902275016996 0.001149748148464743
0.9831532251640601 0.016586537848054042 0.0002602369878858396
0.9836403647717865 0.01659469723326389 -0.00023506200505040223
0.9851303693475361 0.016612178614792795 -0.001742547962328856
0.9833883900528947 0.016597027673593678 1.4582273511632576e-05
0.9816881449450591 0.016569484454684504 0.0017423706002564245
0.98

In [1304]:
def get_swaption_price(mcs, Ti, Tj, K_):
    swaption_price = 0
    P0_Tj = P0[Tj]
    n = len(mcs)
    for i in range(n):
        P_Ti_Tj = mcs[i]['P'][Ti][Tj]
#         print(P_Ti_Tj)
        sum_part = sum([K_*0.25*mcs[i]['P'][Ti][j] for j in range(Ti+1,Tj+1)])
        V = P0_Tj * max(0, 1-P_Ti_Tj-sum_part)/P_Ti_Tj
        swaption_price +=  V * 1.0/n
    return swaption_price

In [1293]:
K = [0.016769451, 0.015996, 0.015720, 0.015988, 0.015533, 0.015418, 0.015566, 0.015336, 0.015186, 0.015237]

In [1305]:
for i, (Ti, Tj) in enumerate([(0,4),(0,8),(0,12),(1,5),(1,9),(1,13),(2,6),(2,10),(3,7),(3,11)]):
#     print("{},{},{}".format(Ti, Tj, get_swaption_price(mcs, Ti, Tj)))
    print(get_swaption_price(mcs, Ti, Tj, K[i]))

0.00015522113938307893
9.671154407768059e-05
0.00019396414437717837
0.00045778782592777645
0.0005232423663551627
0.0004305334614332111
0.00043790259621130125
0.0005093808247931757
0.0003342162051815922
0.000783113131450461


In [1360]:
def get_b_swaption_price(mcs):
    n = len(mcs)
    k_ = 0.015
    Q = [[0 for _ in range(n)] for _ in range(7)]
    for t in range(7, 0, -1):
        X = []
        ee_list = []
        cont_list = []
        for i in range(n):
            # TODO: Calculate Q12 CV and Call Value from all path
#             print(monte_carlo_cache[i]['P'][t-1][-1])
            sum_part = sum([0.25*(mcs[i]['Fk'][t+1][j]-k_) * mcs[i]['P'][t][j] for j in range(t+1, 12)])
            print(sum_part)
            v = max(sum_part, 0)
            q = v / mcs[i]['P'][t][11]
#             V += v,
            print(q)
            Q[t-1][i] = q
            if t == 7: continue
            ee_list += sum_part,
            cont_val = Q[t][i] * mcs[i]['P'][t][11]
            cont_list += cont_val.real,
            x1 = mcs[i]['Fk'][t][t].real
            x2 = mcs[i]['S'][t-1]
            X += [x1, x2, x1**2, x2**2, x1*x2],
        # TODO: Linear Regression get coefficients
        if t == 7: continue
        b0, b1, b2, b3, b4, b5 = get_coef_LR(X, cont_list)
#         print("Time {}, b0: {}, b1: {}, b2: {}, b3: {}, b4: {}, b5: {}".format(t, b0, b1, b2, b3, b4, b5))
        for i in range(n):
            # TODO: Recalculate VC for all path and compare with Call Value
            cont_val_hat = b0 + b1*X[i][0] + b2*X[i][1] + b3*X[i][2] + b4*X[i][3] + b5*X[i][4]
            ee_val = ee_list[i]
            Q[t-1][i] = (ee_val if ee_val > cont_val_hat else cont_list[i]) / mcs[i]['P'][t-1][11]
    return Q

In [1361]:
sss_  = get_b_swaption_price(mcs)

(-0.0012601390212667756-0.0005059838580267107j)
0.0
(0.002008580530003374-0.002254651564895411j)
(0.0020429881473261824-0.0022932744565755682j)
(-0.0011237646409271353+0.002714649178670404j)
0.0
(-0.00021257972608650457-0.00226073832997303j)
0.0
(-0.003507841611034955-0.0009545383529292704j)
0.0
(0.001950803174107711+0.0016325653152222408j)
(0.001984096128227697+0.0016604271327848635j)
(0.0034865840839193346+0.002447562054416771j)
(0.003551622671244209+0.0024932187701527902j)
(-0.001000700766450667-0.003255084887729698j)
0.0
(0.00206893494282285-0.0008931347677383135j)
(0.002104542349795362-0.0009085060645818805j)
(-0.0025640964462959204+0.0009478774399753402j)
0.0
(-0.000941890170900902-0.00107572499163728j)
0.0
(-0.0014533991188075473+0.002763512444404144j)
0.0
(1.4806275410649493e-06+9.552552588258057e-05j)
(1.5029804309915213e-06+9.696766545246552e-05j)
(0.0008997790322599459-0.002372947450237313j)
(0.0009141896704643003-0.002410951989082175j)
(0.003532435538274752-0.00014602357137

In [1362]:
sss_[0]

[0.0019386915668962914,
 0.002091744921745125,
 0.0,
 0.0,
 0.0,
 (0.0038928037517413283+0.002223569685871071j),
 (0.0029690286920769377-0.0025631795053517063j),
 (0.001472031296394147+0.0013018978936225702j),
 (0.0017453770805405796-0.0007890041832278484j),
 0.0,
 (0.002809693843686334-0.0008664289239116545j),
 (0.0017132658609800941+0.0004104992122793279j),
 1.5374876823438595e-06,
 0.0009351265659615911,
 (0.0034987437072709067+0.0030227160883180256j),
 0.0,
 0.0016103893359977332,
 0.0032422814532564693,
 0.0022848123263735374,
 (0.002485260981831155+0.0014608340571700428j),
 0.0009113120399070227,
 (0.004977858988295509-0.0022106143255124573j),
 0.0,
 0.0021635105142601588,
 0.0,
 (0.0018152325184063074+0.0014954910216481649j),
 0.0,
 0.001443219502533572,
 0.0,
 0.0]

In [1364]:
np.mean(sss_[0]).real

0.0014667307478612383